<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 8: Quantum Key Distribution Protocols</h2>

Welcome to the new Julia sheet! As usual, we will ask you to use Julia to answer a few exercises. Most importantly, however, the purpose of these Julia sheets is for you to play around and build intuition by exploring and calculating things we do NOT ask you :-) We hope that you take advantage of using Julia this way.

In this Julia Jupyter notebook, we focus on exploring the BB84 QKD protocol and the quantum bit error rate (qber), as well as its entangled counterpart!

* <a href="#bb84"> BB84 Protocol </a>

* <a href="#e91"> E91 Entanglement Based Protocol </a>


The include command that follows will include all the functions that you have used on the previous chapters.

In [ ]:
include("source/main.jl");

<a id="bb84"></a>
<h2 style="color:#00A6D6;">BB84 QKD</h2>

Let's first explore BB84 QKD, starting with Alice sending BB84 encoded qubits to Bob. When there are no errors, we expect that if Bob measured in the same basis as Alice used,then the outcome measured by Bob is the same as the bit encoded by Alice in a particular round. That is, in the perfect case, we expect the QBER in both bases to be 0.

Play around with the piece of code below. 

In [ ]:
# Let's define the elements of the BB84 Basis as ket vectors
# The first index indicates the basis and the second the basis element
v0 = [1 0]';
v1 = [0 1]';
w0 = ([1 1]')/sqrt(2);
w1 = ([1 -1]')/sqrt(2);

# The BB84 states. Note that due to Julia indexing from 1 onwards, 1 is the standard basis, and 2 the Hadamard one
# Similarly bit x=0 will correspond to index 1, and x=1 to index 2
BB84 = [[v0] [v1]; [w0] [w1]];

# Let's set how long a BB84 string Alice and Bob will produce
n = 100;

# Let's now do BB84!
xA = zeros(Int8,n); 
xB = zeros(Int8,n); 
thetaA = zeros(Int8,n);
thetaB = zeros(Int8,n);
errorCount = [0;0];
sameBasis = [0;0];
diffBasis = 0;
sameOutcomeAnyways = 0;

for i = 1:n

        # Alice picks a random BB84 state
        # Bit of the string
        xA[i] = rand(1:2);
        # Basis to encode in
        thetaA[i] = rand(1:2);
    
    
        # Here Alice would send the state to Bob
        Psi = BB84[thetaA[i],xA[i]][1:2];
    
        # Bob picks a random basis
        thetaB[i] = rand(1:2);
    
        # and measures in that basis
        p0 = round((abs(((BB84[thetaB[i],1][1:2])' * Psi)))^2; digits=10);
        p1 = 1-p0;
    
        # Sample from {0,1} where p1 is given 
        rndDist = Binomial(1,p1);
        outcome = rand(rndDist);
        
        # Print basis used - comment/uncomment as desired
        # print("Alice used basis ", thetaA[i]-1, " with bit ", xA[i]-1," and Bob measured in basis ", thetaB[i]-1, " and got outcome ", outcome, "\n");
    
        # Compute potential errors
        if (thetaA[i] == thetaB[i])
            # Alice and Bob measured in the same basis
            sameBasis[thetaA[i]] = sameBasis[thetaA[i]] + 1;
            if (xA[i] != (outcome+1))
                # this is an error
                errorCount[thetaA[i]] = errorCount[thetaA[i]] + 1;
            end
        else
            # Alice and Bob measured in different basis, we will ignore this in the protocol but print for understanding
            diffBasis = diffBasis + 1;
            if (xA[i] == (outcome+1))
                sameOutcomeAnyways = sameOutcomeAnyways + 1;
            end
        end
end

# Let's print the quantum bit error rate in the standard basis
qberZ = errorCount[1]/sameBasis[1];
print("QBER in the standard basis: ", qberZ, "\n");

qberX = errorCount[2]/sameBasis[2];
print("QBER in the Hadamard basis: ", qberX, "\n");

# For fun, lets also print what happens for the bits not in the same basis
sameDiffBasis = sameOutcomeAnyways/diffBasis;
print("Rate different basis but same outcome: ", sameDiffBasis, "\n");


<h3 style="color:#00A6D6;"> Exercise 1</h3>
Examine what happens during the execution of BB84 using the code given above. 
<ul>
<li> When choosing a small value for $n$, print out what happens in each round by uncommenting the printing line above. What do you observe?</li>
    <li> Why is the QBER in both bases 0 in the case above? Did you expect this?</li>
    <li> Look at the success rate with which Bob nevertheless gets the same outcome, even if they measured in the different basis. What do you observe as you make $n$ large?</li>
 </ul>
        


<h3 style="color:#00A6D6;"> Exercise 2</h3>

Let's now imagine that the quantum communication channel from Alice to Bob is not noise free, but that an error occurs. Specifically, let's imagine that we have one of the following two errors:
<ul> 
    <li>With probability $q$, a phase flip $Z$ is applied to the state sent by Alice.</li>
    <li> With probability $q$, a bit flip $X$ is applied to the state sent by Alice</li>
   </ul>
Adapt the code above to investigate what happens in each of the two cases. What do you observe for the QBER in the standard and Hadamard basis? Is this what you expected? To do so, the example code of how to introduce errors may be useful.

In [ ]:
# phase flip Z
Z = [1 0;0 -1];

# bit flip X
X = [0 1;1 0];

# Let's see how we could simulate a noisy channel that applies an error matrix with probability q to an example state
Psi = BB84[1,1][1:2];
errPsi = Psi;
q = 0.9 # error probability
errorRnd = Bernoulli(q);
error = rand(errorRnd);
if error
    errPsi = X * Psi;
end
print("Input state was ",Psi," output state is ", errPsi, "\n");

In [ ]:
# Your code goes here, where you may want to start by copy pasting the BB84 code above and adapting it.

In the book, we saw that the asymptotic key rate in BB84 (using one way discussion) is given by $1-h(qber)$, where $qber$ is the quantum bit error rate. This means that asymptotically, we could hope to obtain $n (1-h(qber))$ key bits from a raw key of size $n$.

In [ ]:
# Compute the binary entropy
function binEntropy(p) 
    entropy = -p * log2(p) - (1-p) * log2(1-p);
    return entropy
end

# Compute the asymptotic secret key length given n and QBER p
function secretKey(n,p)
    skr = n * (1 - binEntropy(p));
    return skr
end

p = range(0,1, length=100);
binEnt = binEntropy.(p);
plot(p,binEnt)
xlabel("p")
ylabel("h(p)")
PyPlot.title("Binary entropy")
grid("on")




<h3 style="color:#00A6D6;">Exercise 3</h3>
Let's investigate the binary entropy function in terms of the qber. For what values of $qber$ can we hope to produce key at all?

In [ ]:
# Your code goes here

<a id="e91"></a>
<h2 style="color:#00A6D6;">E91 Entanglement Based QKD</h2>

Let us now look at the protocol, where Alice and Bob share an EPR pair and make measurements in random basis (standard/Hadamard). This was discovered independently as E91 and paved the way for proving the security of QKD, which is difficult in the prepare and measure version of BB84. Nevertheless, as we saw in the book, the two are in fact equivalent! 

Let us now first explore the E91 version using the code given below.

In [ ]:
# Let's define the elements of the BB84 Basis as ket vectors
# The first index indicates the basis and the second the basis element
v0 = [1 0]';
v1 = [0 1]';
w0 = ([1 1]')/sqrt(2);
w1 = ([1 -1]')/sqrt(2);

# The BB84 states. Note that due to Julia indexing from 1 onwards, 1 is the standard basis, and 2 the Hadamard one
# Similarly bit x=0 will correspond to index 1, and x=1 to index 2
BB84 = [[v0] [v1]; [w0] [w1]];

# Entangled input state they will use
epr = ([1 0 0 1])/sqrt(2);
rhoAB = epr' * epr;

# Build the basis measurements on Alice and Bob's side, we will keep them around for potential future use
# We will also compute the probabilities of outcomes immediately
measAB = Dict();
outcomeProb = Dict();
for thetaA = 1:2
    for thetaB = 1:2
        outcomeProb[(thetaA,thetaB)] = zeros(4);
        pcount = 1;
        for xA = 1:2
            for xB = 1:2
                a = BB84[thetaA,xA][1:2];
                b = BB84[thetaB,xB][1:2]; 

                # Construct the measurement operator on A and B
                measAB[(thetaA,xA,thetaB,xB)] = kron(a*a', b*b');

                # compute the outcome probability on rho that we will sample from later, and add it to a probability vector
                outcomeProb[(thetaA,thetaB)][pcount] = tr(rhoAB *  measAB[(thetaA,xA,thetaB,xB)]);
                pcount = pcount + 1;
            end
        end
    end
end


# Let's set how long a BB84 string Alice and Bob will produce
n = 100;

# Let's now do BB84!
xA = zeros(Int8,n); 
xB = zeros(Int8,n); 
thetaA = zeros(Int8,n);
thetaB = zeros(Int8,n);
errorCount = [0;0];
sameBasis = [0;0];
diffBasis = 0;
sameOutcomeAnyways = 0;

for i = 1:n
  
        # Alice picks a random basis
        thetaA[i] = rand(1:2);
    
        # Bob picks a random basis
        thetaB[i] = rand(1:2);
    
        # Sample from the outcome distribution
        # outcomes mean 1 -> 00, 2 -> 01, 3 -> 10, 4 -> 11
        rng = Multinomial(1,outcomeProb[(thetaA[i],thetaB[i])]);
        outcome = rand(rng);
        if outcome[1] == 1
            aliceOutcome = 0;
            bobOutcome = 0;
        elseif outcome[2] == 1
            aliceOutcome = 0;
            bobOutcome = 1;
        elseif outcome[3] == 1
            aliceOutcome = 1;
            bobOutcome = 0;
        elseif outcome[4] == 1
            aliceOutcome = 1;
            bobOutcome = 1;
        end
        
        # Print basis used - comment/uncomment as desired
        # print("Alice used basis ", thetaA[i]-1, " with bit ", aliceOutcome ," and Bob measured in basis ", thetaB[i]-1, " and got outcome ", bobOutcome, "\n");
    
        # Compute potential errors
        if (thetaA[i] == thetaB[i])
            # Alice and Bob measured in the same basis
            sameBasis[thetaA[i]] = sameBasis[thetaA[i]] + 1;
            if (aliceOutcome != bobOutcome)
                # this is an error
                errorCount[thetaA[i]] = errorCount[thetaA[i]] + 1;
            end
        else
            # Alice and Bob measured in different basis, we will ignore this in the protocol but print for understanding
            diffBasis = diffBasis + 1;
            if (aliceOutcome == bobOutcome)
                sameOutcomeAnyways = sameOutcomeAnyways + 1;
            end
        end
end

# Let's print the quantum bit error rate in the standard basis
qberZ = errorCount[1]/sameBasis[1];
print("QBER in the standard basis: ", qberZ, "\n");

qberX = errorCount[2]/sameBasis[2];
print("QBER in the Hadamard basis: ", qberX, "\n");

# For fun, lets also print what happens for the bits not in the same basis
sameDiffBasis = sameOutcomeAnyways/diffBasis;
print("Rate different basis but same outcome: ", sameDiffBasis, "\n");


<h3 style="color:#00A6D6;"> Exercise 4</h3>
Let's examine again what happens for the following questions that you already took at look at in Exericse 1:
<ul>
<li> When choosing a small value for $n$, print out what happens in each round by uncommenting the printing line above. What do you observe?</li>
    <li> Why is the QBER in both bases 0 in the case above? Did you expect this?</li>
    <li> Look at the success rate with which Bob nevertheless gets the same outcome, even if they measured in the different basis. What do you observe as you make $n$ large?</li>
 </ul>
        

<h3 style="color:#00A6D6;"> Exercise 5</h3>

Let's again imagine that errors occur. Imagine that either:
<ul> 
    <li>With probability $q$, a phase flip $Z$ is applied to Alice's qubit before measuring.</li>
    <li> With probability $q$, a bit flip $X$ is applied to Alice's qubit before measuring</li>
   </ul>
Adapt the code above to investigate what happens in each of the two cases. What do you observe for the QBER in the standard and Hadamard basis? Is this what you expected? To do so, the example code of how to introduce errors may be useful.

In [ ]:
# Your code goes here.

<h3 style="color:#00A6D6;"> Exercise 6</h3>
Repeat the exercise above, but this time apply the noise to Bob's qubit. Does this matter? And, is this what you expected?

In [ ]:
# Your code goes here.